# Import Libraries and Classes

In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix

# Read the dataset

In [42]:
dataset = pd.read_csv('datasets/Titanic-Dataset.csv')

In [28]:
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Finding if null values exists in some columns

In [43]:
dataset[['Age', 'Fare']].isnull().sum()


Age     177
Fare      0
dtype: int64

# Finding if Cabin and Embarked columns are a unique values or not

In [44]:
dataset['Cabin'].value_counts()

Cabin
B96 B98        4
G6             4
C23 C25 C27    4
C22 C26        3
F33            3
              ..
E34            1
C7             1
C54            1
E36            1
C148           1
Name: count, Length: 147, dtype: int64

In [45]:
dataset['Embarked'].value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

# Split dataset into Train/Val/Test

In [46]:
x = dataset.drop(['Survived'], axis=1)
y = dataset['Survived']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42, stratify=y)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=42, stratify=y_test)



# Dividing Categorical and Numerical Columns

In [47]:
numeric_features = ['Pclass', 'SibSp', 'Parch', 'Age', 'Fare']
categorical_features = ['Sex', 'Cabin', 'Embarked']

# pipeline for numerical data

In [48]:
numerical_pipeline = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler()
)

# Pipeline for categorical data

In [49]:
categorical_pipeline = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OneHotEncoder(handle_unknown='ignore')
)

# Combining both pipelines

In [50]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_pipeline, numeric_features),
        ("cat", categorical_pipeline, categorical_features)
    ]
)   

# Model with class balancing

In [51]:
rf = RandomForestClassifier(class_weight= 'balanced', n_estimators=100, random_state=42)

# Final pipeline with model

In [52]:
pipeline = make_pipeline(
    preprocessor,
    rf
)

# Train and evaluate

In [53]:
pipeline.fit(x_train, y_train)
y_pred = pipeline.predict(x_val)

print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.90      0.87        82
           1       0.83      0.73      0.78        52

    accuracy                           0.84       134
   macro avg       0.83      0.82      0.82       134
weighted avg       0.84      0.84      0.83       134

[[74  8]
 [14 38]]
